In [1]:
!pip install neo4j
from yfiles_jupyter_graphs import GraphWidget
! pip install langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate


In [2]:
from neo4j import GraphDatabase

class DataLoader:
    def __init__(self, uri, user, password, llm):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        self.graph = Neo4jGraph(url=uri, username=user, password=password)
        self.llm = llm
        self.cypher_generation_prompt = self.create_cypher_generation_prompt()
        self.chain = None  # Initialize to None, will be set in create_chain()

    def close(self):
        self._driver.close()

    def run_query(self, query, params=None):
        with self._driver.session() as session:
            session.run(query, params)
            
    def run_query_and_return(self, query, params=None):
        with self._driver.session() as session:
            result = session.run(query, params)
            return [record for record in result]
        
    def run_query_and_return_graph(self, query, params=None):
        with self._driver.session() as session:
            result = session.run(query, params)
            return result.graph()

    def get_schema(self):
        def transaction(tx):
            result = tx.run("CALL db.schema.visualization()")
            return result.data()

        with self._driver.session() as session:
            schema_data = session.read_transaction(transaction)
            return schema_data

    def create_cypher_generation_prompt(self):
        cypher_generation_template = """Task:Generate Cypher statement to query a graph database.
        Instructions:
        Use only the provided relationship types and properties in the schema.
        Do not use any other relationship types or properties that are not provided.
        Schema:
        {schema}
        Note: Do not include any explanations or apologies in your responses.
        Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
        Do not include any text except the generated Cypher statement.

        The question is:
        {question}"""

        return PromptTemplate(
            input_variables=["schema", "question"], template=cypher_generation_template
        )

    def create_chain(self):
        self.chain = GraphCypherQAChain.from_llm(
            llm=self.llm,
            graph=self.graph,
            verbose=True,
            cypher_prompt=self.cypher_generation_prompt
        )

    def run_chain_query(self, query):
        if self.chain is None:
            self.create_chain()
        return self.chain.run(query)

# Usage:
uri = 'bolt://localhost:7687'
username = 'neo4j'
password = ''
openai_api_key = ""
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)

# Initialize the DataLoader
loader = DataLoader(uri, username, password, llm)


# Don't forget to close the connection when done
# loader.close()

# Set parameters
params = {
    "file_path_root": "file:///",
    "idsToSkip": []
}

# Run the constraint creation queries
constraint_queries = [
    "CREATE CONSTRAINT `imp_uniq_Territory_territoryID` IF NOT EXISTS FOR (n: `Territory`) REQUIRE (n.`territoryID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Region_regionID` IF NOT EXISTS FOR (n: `Region`) REQUIRE (n.`regionID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Supplier_supplierID` IF NOT EXISTS FOR (n: `Supplier`) REQUIRE (n.`supplierID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Shipper_shipperID` IF NOT EXISTS FOR (n: `Shipper`) REQUIRE (n.`shipperID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Product_productID` IF NOT EXISTS FOR (n: `Product`) REQUIRE (n.`productID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Category_categoryID` IF NOT EXISTS FOR (n: `Category`) REQUIRE (n.`categoryID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Order_orderID` IF NOT EXISTS FOR (n: `Order`) REQUIRE (n.`orderID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Customer_customerID` IF NOT EXISTS FOR (n: `Customer`) REQUIRE (n.`customerID`) IS UNIQUE",
    "CREATE CONSTRAINT `imp_uniq_Employee_employeeID` IF NOT EXISTS FOR (n: `Employee`) REQUIRE (n.`employeeID`) IS UNIQUE"
]

for query in constraint_queries:
    loader.run_query(query)


In [3]:
# List of queries for loading each type of node
queries = [
    # Load Territories
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "territories.csv" AS row RETURN row',
        'MERGE (n:Territory {territoryID: row.territoryID}) SET n.territoryDescription = row.territoryDescription',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load Regions
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "regions.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`regionID` IS NULL AND NOT row.`regionID` IN $idsToSkip MERGE (n:Region {regionID: row.regionID}) SET n.regionDescription = row.regionDescription',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Suppliers
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "suppliers.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`supplierID` IS NULL AND NOT row.`supplierID` IN $idsToSkip MERGE (n:Supplier {supplierID: row.supplierID}) SET n.companyName = row.companyName, n.contactName = row.contactName, n.contactTitle = row.contactTitle, n.address = row.address, n.city = row.city, n.region = row.region, n.postalCode = row.postalCode, n.country = row.country, n.phone = row.phone, n.fax = row.fax, n.homePage = row.homePage',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Shippers
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "shippers.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`shipperID` IS NULL AND NOT row.`shipperID` IN $idsToSkip MERGE (n:Shipper {shipperID: row.shipperID}) SET n.companyName = row.companyName, n.phone = row.phone',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Products
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "products.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`productID` IS NULL AND NOT row.`productID` IN $idsToSkip MERGE (n:Product {productID: row.productID}) SET n.productName = row.productName, n.quantityPerUnit = toInteger(trim(row.quantityPerUnit)), n.unitPrice = toFloat(trim(row.unitPrice)), n.unitsInStock = toInteger(trim(row.unitsInStock)), n.unitsOnOrder = toInteger(trim(row.unitsOnOrder)), n.reorderLevel = toInteger(trim(row.reorderLevel)), n.discontinued = toLower(trim(row.discontinued)) IN ["1", "true", "yes"]',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Categories
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "categories.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`categoryID` IS NULL AND NOT row.`categoryID` IN $idsToSkip MERGE (n:Category {categoryID: row.categoryID}) SET n.categoryName = row.categoryName, n.description = row.description',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Orders
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "orders.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`orderID` IS NULL AND NOT row.`orderID` IN $idsToSkip MERGE (n:Order {orderID: row.orderID}) SET n.orderDate = row.orderDate, n.requiredDate = row.requiredDate, n.shippedDate = row.shippedDate, n.freight = row.freight, n.shipName = row.shipName, n.shipAddress = row.shipAddress, n.shipCity = row.shipCity, n.shipRegion = row.shipRegion, n.shipPostalCode = row.shipPostalCode, n.shipCountry = row.shipCountry',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Customers
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "customers.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`customerID` IS NULL AND NOT row.`customerID` IN $idsToSkip MERGE (n:Customer {customerID: row.customerID}) SET n.companyName = row.companyName, n.contactName = row.contactName, n.contactTitle = row.contactTitle, n.address = row.address, n.city = row.city, n.region = row.region, n.postalCode = row.postalCode, n.country = row.country, n.phone = row.phone, n.fax = row.fax',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    ''',
    # Load Employees
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "employees.csv" AS row RETURN row',
        'WITH row WHERE NOT row.`employeeID` IS NULL AND NOT row.`employeeID` IN $idsToSkip MERGE (n:Employee {employeeID: row.employeeID}) SET n.lastName = row.lastName, n.firstName = row.firstName, n.title = row.title, n.titleOfCourtesy = row.titleOfCourtesy, n.birthDate = row.birthDate, n.hireDate = row.hireDate, n.address = row.address, n.city = row.city, n.region = row.region, n.postalCode = row.postalCode, n.country = row.country, n.homePhone = row.homePhone, n.extension = row.extension, n.notes = row.notes, n.photoPath = row.photoPath',
        {batchSize: 100, params: {file: $file_path_root, idsToSkip: $idsToSkip}, parallel: true}
    )
    '''
]

# Run each query
for query in queries:
    loader.run_query(query, params)



In [4]:
relationship_queries = [
    # Load IN_REGION relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "territories.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Territory {territoryID: row.territoryID})
         MATCH (target:Region {regionID: row.regionID})
         MERGE (source)-[r:IN_REGION]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load SUPPLIES relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "products.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Supplier {supplierID: row.supplierID})
         MATCH (target:Product {productID: row.productID})
         MERGE (source)-[r:SUPPLIES]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load PART_OF relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "products.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Product {productID: row.productID})
         MATCH (target:Category {categoryID: row.categoryID})
         MERGE (source)-[r:PART_OF]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load SHIPS relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "orders.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Shipper {shipperID: row.shipVia})
         MATCH (target:Order {orderID: row.orderID})
         MERGE (source)-[r:SHIPS]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load PURCHASED relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "orders.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Customer {customerID: row.customerID})
         MATCH (target:Order {orderID: row.orderID})
         MERGE (source)-[r:PURCHASED]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load SOLD relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "orders.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Employee {employeeID: row.employeeID})
         MATCH (target:Order {orderID: row.orderID})
         MERGE (source)-[r:SOLD]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load REPORTS_TO relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "employees.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Employee {employeeID: row.employeeID})
         MATCH (target:Employee {employeeID: row.reportsTo})
         MERGE (source)-[r:REPORTS_TO]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load IN_TERRITORY relationships
    '''
    CALL apoc.periodic.iterate(
        'LOAD CSV WITH HEADERS FROM $file + "employee-territories.csv" AS row RETURN row',
        'WITH row 
         MATCH (source:Employee {employeeID: row.employeeID})
         MATCH (target:Territory {territoryID: row.territoryID})
         MERGE (source)-[r:IN_TERRITORY]->(target)',
        {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    ''',
    # Load ORDERS relationships
    '''
    CALL apoc.periodic.iterate(
      'LOAD CSV WITH HEADERS FROM $file + "order-details.csv" AS row RETURN row',
      'MATCH (source:Order {orderID: row.orderID}) 
       MATCH (target:Product {productID: row.productID}) 
       MERGE (source)-[r:ORDERS]->(target) 
       SET r.unitPrice = toFloat(trim(row.unitPrice)),
           r.quantity = toInteger(trim(row.quantity)),
           r.discount = toFloat(trim(row.discount))',
      {batchSize: 100, params: {file: $file_path_root}, parallel: true}
    )
    '''
]


# Run each relationship query
for query in relationship_queries:
    loader.run_query(query, params)


In [5]:
# Additional direct relationship queries
direct_relationship_queries = [
    # Create direct relationship from Customer to Category
    '''
    MATCH (c:Customer)-[:PURCHASED]->(:Order)-[:ORDERS]->(:Product)-[:PART_OF]->(cat:Category)
    MERGE (c)-[:BUYS_FROM_CATEGORY]->(cat)
    ''',
    
    # Create direct relationship from Customer to Product bypassing the mediating Order nodes
    '''
    MATCH (c:Customer)-[:PURCHASED]->(:Order)-[:ORDERS]->(p:Product)
    MERGE (c)-[:BUYS_PRODUCT]->(p)
    '''
    ,
     # Convert the customer-product-customer to weighted customer - customer
    '''
    MATCH (c1:Customer)-[:BUYS_PRODUCT]->(p:Product)<-[:BUYS_PRODUCT]-(c2:Customer)
    WHERE c1 <> c2 AND id(c1) < id(c2)
    WITH c1, c2, COUNT(DISTINCT p) AS sharedProducts
    MERGE (c1)-[r:SHARED_PURCHASE]->(c2)
    SET r.weight = sharedProducts
    '''
]


# Run each direct relationship query
for query in direct_relationship_queries:
    loader.run_query(query, {})

In [6]:

# Query to project the graph for community detection
project_graph_query = '''
CALL gds.graph.project(
    'customer_community_graph',
    {
        Customer: {
            label: 'Customer',
            properties: {}
        }
    },
    {
        SHARED_PURCHASE: {
            type: 'SHARED_PURCHASE',
            properties: 'weight',
            orientation: 'UNDIRECTED'
        }
    }
)
'''

# Run the query
loader.run_query(project_graph_query, {})

In [7]:
# Query to run the Louvain algorithm
louvain_algorithm_query = '''
CALL gds.louvain.write('customer_community_graph', {
    relationshipWeightProperty: 'weight',
    writeProperty: 'community'
})
'''

# Run the query
loader.run_query(louvain_algorithm_query, {})

In [8]:
# Query for community analysis
community_analysis_query = '''
MATCH (c:Customer)
WHERE c.community IS NOT NULL
RETURN c.community as Community, COUNT(*) as Size 
ORDER BY Size DESC
'''

# Run the query and get the results
results = loader.run_query_and_return(community_analysis_query, {})


In [9]:
# Loading the schema of our Graph
schema = loader.run_query_and_return_graph("CALL db.schema.visualization()")

In [10]:
# Visualization of the graph
GraphWidget(graph=schema)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [12]:
import pandas as pd
data = []
for record in results:
    data.append({"Community": record['Community'], "Size": record['Size']})
df = pd.DataFrame(data)
df


,Community,Size
0,33,12
1,70,12
2,88,12
3,34,8
4,82,6
5,0,1
6,2,1
7,3,1
8,4,1
9,6,1


In [13]:
# Query to explore specific communities with community ID = 33
graph = loader.run_query_and_return_graph("MATCH (c:Customer) WHERE c.community = 33 RETURN c")

In [14]:
# Visualization of the graph
GraphWidget(graph=graph)

GraphWidget(layout=Layout(height='500px', width='100%'))

# RECOMMENDER SYSTEM

In [15]:
# Query to create in-memory projection of subgraph "customer product"
project_graph_query1 = '''
CALL gds.graph.project(
    'customer_product_graph',
    {
        Customer: {
            label: 'Customer',
            properties: {}
        },
        Product: {
            label: 'Product',
            properties: {}
        }
    },
    {
        BUYS_PRODUCT: {
            type: 'BUYS_PRODUCT',
            orientation: 'UNDIRECTED',
            properties: {}
        }
    }
)
'''

# Run the query
loader.run_query(project_graph_query1, {})

In [16]:
# Initially we run the Query for Node2Vec Embedding
node2vec_query = '''
CALL gds.beta.node2vec.write('customer_product_graph',
  {
    embeddingDimension: 10,
    walkLength: 80,
    randomSeed: 42,
    writeProperty: 'node2vec_embedding'
  }
)
'''

# Run the Node2Vec query
loader.run_query(node2vec_query)

In [17]:
import pandas as pd
# Query to get the embeddings
get_embeddings_query = '''
MATCH (n:Customer)
RETURN n.companyName, n.node2vec_embedding
LIMIT 10;
'''

# Run the query and get the results
results = loader.run_query_and_return(get_embeddings_query, {})

# Print the embeddings
# Convert the results to a list of dictionaries
records = [dict(record) for record in results]

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(records)

# Display the DataFrame in Jupyter Notebook
df


,n.companyName,n.node2vec_embedding
0,Alfreds Futterkiste,"[0.014049062505364418, -0.001962334383279085, ..."
1,Ana Trujillo Emparedados y helados,"[-0.1506243199110031, -0.7683464884757996, -0...."
2,Antonio Moreno Taquería,"[0.01399531401693821, -0.0002356968616368249, ..."
3,Around the Horn,"[0.013986357487738132, 0.01671874336898327, 0...."
4,Berglunds snabbköp,"[0.014013231731951237, -0.0341445729136467, 0...."
5,Blauer See Delikatessen,"[-0.10955160856246948, -0.5342106223106384, -1..."
6,Blondesddsl père et fils,"[0.013959481380879879, -0.032417938113212585, ..."
7,Bólido Comidas preparadas,"[0.013950524851679802, -0.01546349748969078, 0..."
8,Bon app',"[0.013977399095892906, 0.033673182129859924, -..."
9,Bottom-Dollar Markets,"[-0.05125097930431366, -1.2652692794799805, -0..."


In [18]:
# Query to get the embeddings
get_embeddings_query = '''
MATCH (p:Product)
RETURN p.productName, p.node2vec_embedding
LIMIT 10;
'''

# Run the query and get the results
results = loader.run_query_and_return(get_embeddings_query, {})

# Print the embeddings
# Convert the results to a list of dictionaries
records = [dict(record) for record in results]

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(records)

# Display the DataFrame in Jupyter Notebook
df

,p.productName,p.node2vec_embedding
0,Chai,"[-0.17347189784049988, -1.37179696559906, -0.3..."
1,Chang,"[0.1381937563419342, -1.3769621849060059, -0.8..."
2,Aniseed Syrup,"[-0.08183262497186661, -1.1888643503189087, 0...."
3,Chef Anton's Cajun Seasoning,"[0.1246376484632492, -1.7254425287246704, -0.2..."
4,Chef Anton's Gumbo Mix,"[0.1334913820028305, -1.091832160949707, -0.32..."
5,Grandma's Boysenberry Spread,"[-0.48367130756378174, -0.8422445058822632, -0..."
6,Uncle Bob's Organic Dried Pears,"[-0.02449006959795952, -0.9355024695396423, -0..."
7,Northwoods Cranberry Sauce,"[0.024189669638872147, 0.005612008273601532, -..."
8,Mishi Kobe Niku,"[0.024180712178349495, 0.022566447034478188, 0..."
9,Ikura,"[-0.10931035876274109, -0.7461543083190918, -0..."


In [19]:
knn_query = '''
CALL gds.knn.write('customer_product_graph', {
    topK: 5,
    nodeProperties: ['node2vec_embedding'],
    randomSeed: 42,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0,
    writeRelationshipType: "node2vec_SIMILAR",
    writeProperty: "score"
})
'''

# Run the query
loader.run_query(knn_query, {})

In [20]:
import pandas as pd

# Query to find similar customers based on Node2Vec embeddings
similar_customers_query = '''
MATCH (n:Customer)-[r:node2vec_SIMILAR]->(m:Customer)
WHERE id(n) < id(m) OR (id(n) = id(m) AND n.companyName < m.companyName) 
RETURN n.companyName as Customer1, m.companyName as Customer2, r.score as similarity
ORDER BY similarity DESC, Customer1, Customer2
'''

# Run the query and get the results
results = loader.run_query_and_return(similar_customers_query, {})

# Convert the results to a Pandas DataFrame
df = pd.DataFrame([dict(record) for record in results])

# Display the DataFrame in Jupyter Notebook
df


,Customer1,Customer2,similarity
0,Blauer See Delikatessen,Spécialités du monde,0.989601
1,Blauer See Delikatessen,Spécialités du monde,0.989601
2,Blauer See Delikatessen,Spécialités du monde,0.989601
3,Blauer See Delikatessen,Spécialités du monde,0.989601
4,Blauer See Delikatessen,Spécialités du monde,0.989601
...,...,...,...
609,Lazy K Kountry Store,Queen Cozinha,0.657635
610,Océano Atlántico Ltda.,Rancho grande,0.657403
611,Antonio Moreno Taquería,Gourmet Lanchonetes,0.657118
612,Let's Stop N Shop,Santé Gourmet,0.637836


In [21]:
# Query to identify products purchased by similar customers based on a certain similarity threshold
products_purchased_query = '''
MATCH (target:Customer {companyName: "Blauer See Delikatessen"})-[r:node2vec_SIMILAR]->(similar:Customer)
WHERE r.score > 0.8
MATCH (similar)-[:BUYS_PRODUCT]->(product:Product)-[:PART_OF]->(category:Category)
WHERE NOT EXISTS((target)-[:BUYS_PRODUCT]->(product))
RETURN product.productName AS RecommendedProduct, 
       category.categoryName AS ProductCategory,
       COUNT(DISTINCT similar) AS NumOfSimilarCustomers
ORDER BY NumOfSimilarCustomers DESC, RecommendedProduct, ProductCategory
LIMIT 10
'''

# Run the query and get the results
results = loader.run_query_and_return(products_purchased_query, {})

# Convert the results to a Pandas DataFrame
df = pd.DataFrame([dict(record) for record in results])

# Display the DataFrame in Jupyter Notebook
df


,RecommendedProduct,ProductCategory,NumOfSimilarCustomers
0,Alice Mutton,Meat/Poultry,2
1,Chai,Beverages,2
2,Côte de Blaye,Beverages,2
3,Flotemysost,Dairy Products,2
4,Gorgonzola Telino,Dairy Products,2
5,Raclette Courdavault,Dairy Products,2
6,Steeleye Stout,Beverages,2
7,Boston Crab Meat,Seafood,1
8,Chang,Beverages,1
9,Chef Anton's Cajun Seasoning,Condiments,1


In [22]:
# Query to identify the most preferred category for a specific customer
preferred_category_query = '''
MATCH (customer:Customer {companyName: "Blauer See Delikatessen"})-[:BUYS_PRODUCT]->(:Product)-[:PART_OF]->(category:Category)
WITH category, COUNT(*) AS productsPurchasedCount
ORDER BY productsPurchasedCount DESC
LIMIT 1
WITH category.categoryName AS topCategory
RETURN topCategory
'''

# Run the query and get the results
results = loader.run_query_and_return(preferred_category_query, {})

# Convert the results to a Pandas DataFrame
df = pd.DataFrame([dict(record) for record in results])

# Display the DataFrame in Jupyter Notebook
df


,topCategory
0,Dairy Products


In [23]:
# Query to prioritize recommendations based on the most preferred category
prioritized_recommendations_query = '''
WITH "Blauer See Delikatessen" AS target_customer_name, 0.8 AS score_threshold
MATCH (customer:Customer {companyName: target_customer_name})-[:BUYS_PRODUCT]->(:Product)-[:PART_OF]->(category:Category)
WITH category, COUNT(*) AS productsPurchasedCount, target_customer_name, score_threshold
ORDER BY productsPurchasedCount DESC
LIMIT 1
WITH category.categoryName AS topCategory, target_customer_name, score_threshold

MATCH (target:Customer {companyName: target_customer_name})-[r:node2vec_SIMILAR]->(similar:Customer)
WHERE r.score > score_threshold
MATCH (similar)-[:BUYS_PRODUCT]->(product:Product)-[:PART_OF]->(category:Category)
WHERE NOT EXISTS((target)-[:BUYS_PRODUCT]->(product))
WITH product, category, COUNT(DISTINCT similar) AS NumOfSimilarCustomers, topCategory
ORDER BY CASE WHEN category.categoryName = topCategory THEN 1 ELSE 0 END DESC,
         NumOfSimilarCustomers DESC,
         product.productName
LIMIT 10
RETURN product.productName AS RecommendedProduct,
       category.categoryName AS ProductCategory,
       NumOfSimilarCustomers;
'''

# Run the query and get the results
results = loader.run_query_and_return(prioritized_recommendations_query, {})

# Convert the results to a Pandas DataFrame
df = pd.DataFrame([dict(record) for record in results])

# Display the DataFrame in Jupyter Notebook
df


,RecommendedProduct,ProductCategory,NumOfSimilarCustomers
0,Flotemysost,Dairy Products,2
1,Gorgonzola Telino,Dairy Products,2
2,Raclette Courdavault,Dairy Products,2
3,Gudbrandsdalsost,Dairy Products,1
4,Mozzarella di Giovanni,Dairy Products,1
5,Alice Mutton,Meat/Poultry,2
6,Chai,Beverages,2
7,Côte de Blaye,Beverages,2
8,Steeleye Stout,Beverages,2
9,Boston Crab Meat,Seafood,1


In [24]:
# Run chain query
chain_query_result = loader.run_chain_query("How many people bought Chai")
print(chain_query_result)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer)-[:BUYS_PRODUCT]->(p:Product)
WHERE p.productName = 'Chai'
RETURN COUNT(c)
Full Context:
[{'COUNT(c)': 22}]

> Finished chain.
22 people bought Chai.


In [26]:
chain_query_result2 = loader.run_chain_query("I want to give me all distinct products in a column with their price in second columne")
print(chain_query_result2)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)
RETURN DISTINCT p.productName AS Product, p.unitPrice AS Price
Full Context:
[{'Product': 'Chai', 'Price': 18.0}, {'Product': 'Chang', 'Price': 19.0}, {'Product': 'Aniseed Syrup', 'Price': 10.0}, {'Product': "Chef Anton's Cajun Seasoning", 'Price': 22.0}, {'Product': "Chef Anton's Gumbo Mix", 'Price': 21.35}, {'Product': "Grandma's Boysenberry Spread", 'Price': 25.0}, {'Product': "Uncle Bob's Organic Dried Pears", 'Price': 30.0}, {'Product': 'Northwoods Cranberry Sauce', 'Price': 40.0}, {'Product': 'Mishi Kobe Niku', 'Price': 97.0}, {'Product': 'Ikura', 'Price': 31.0}]

> Finished chain.
Sure! Here are all the distinct products along with their corresponding prices:

1. Chai - $18.0
2. Chang - $19.0
3. Aniseed Syrup - $10.0
4. Chef Anton's Cajun Seasoning - $22.0
5. Chef Anton's Gumbo Mix - $21.35
6. Grandma's Boysenberry Spread - $25.0
7. Uncle Bob's Organic Dried Pears - $30.0
8. Northwoods Cranberry Sau

In [28]:
# Query to explore specific communities with community ID = 33
graph = loader.run_query_and_return_graph("MATCH (p:Product)  RETURN p")

In [29]:
# Visualization of the graph
GraphWidget(graph=graph)

GraphWidget(layout=Layout(height='500px', width='100%'))